In [3]:
!pip install nltk rouge-score scikit-learn torch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [4]:
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
from Datatests import *

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
df = pd.read_csv('/content/gdrive/My Drive/Indigo-HackToHire/processed_data.csv', index_col=0)
df.head(5)

,question,tokenized_question,stemmed_question,setmmed_nostop_answer,answer,tokenized_answer,stemmed_answer
0,Why whenever I get in the shower my girlfriend...,"['why', 'whenever', 'i', 'get', 'in', 'the', '...","['whi', 'whenev', 'i', 'get', 'in', 'the', 'sh...","['whi', 'whenev', 'get', 'shower', 'girlfriend...",Isn’t it awful? You would swear that there was...,"['isn’t', 'it', 'awful', 'you', 'would', 'swea...","['isn’t', 'it', 'aw', 'you', 'would', 'swear',..."
1,"What is a proxy, and how can I use one?","['what', 'is', 'a', 'proxy', 'and', 'how', 'ca...","['what', 'is', 'a', 'proxi', 'and', 'how', 'ca...","['proxi', 'use', 'one']",A proxy server is a system or router that prov...,"['a', 'proxy', 'server', 'is', 'a', 'system', ...","['a', 'proxi', 'server', 'is', 'a', 'system', ..."
2,"What song has the lyrics ""someone left the cak...","['what', 'song', 'has', 'the', 'lyrics', 'some...","['what', 'song', 'ha', 'the', 'lyric', 'someon...","['song', 'ha', 'lyric', 'someon', 'left', 'cak...",MacArthur's Park\n,"['macarthur', 's', 'park']","['macarthur', 's', 'park']"
3,I am the owner of an adult website called http...,"['i', 'am', 'the', 'owner', 'of', 'an', 'adult...","['i', 'am', 'the', 'owner', 'of', 'an', 'adult...","['owner', 'adult', 'websit', 'call', 'http', '...",Don't let apps that are liers put adds on your...,"['don', 't', 'let', 'apps', 'that', 'are', 'li...","['don', 't', 'let', 'app', 'that', 'are', 'lie..."
4,Does the Bible mention anything about a place ...,"['does', 'the', 'bible', 'mention', 'anything'...","['doe', 'the', 'bibl', 'mention', 'anyth', 'ab...","['doe', 'bibl', 'mention', 'anyth', 'place', '...",St. John in the book of Revelation mentions an...,"['st', 'john', 'in', 'the', 'book', 'of', 'rev...","['st', 'john', 'in', 'the', 'book', 'of', 'rev..."


In [7]:
# Test & Train Split
df = df.sample(frac=1)

# Ratio and index definition
ratio = 0.75
total_rows = df.shape[0]
train_size = int(total_rows*ratio)

# Split data into test and train
train = df[0:train_size]
test = df[train_size:]

In [ ]:
# Initialize tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Prepare dataset and dataloader
train_dataset = T5Dataset(df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/gdrive/My Drive/Indigo-HackToHire/T5results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='/content/gdrive/My Drive/Indigo-HackToHire/logs',
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Train the model
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss
500,0.377600
1000,0.023700
1500,0.013300
2000,0.007900
2500,0.008200
3000,0.006700
3500,0.004200
4000,0.003800
4500,0.004000
5000,0.002700


Step,Training Loss
500,0.377600
1000,0.023700
1500,0.013300
2000,0.007900
2500,0.008200
3000,0.006700
3500,0.004200
4000,0.003800
4500,0.004000
5000,0.002700


In [ ]:
# Save the model
model.save_pretrained('/content/gdrive/My Drive/Indigo-HackToHire/t5_model')
tokenizer.save_pretrained('/content/gdrive/My Drive/Indigo-HackToHire/t5_model')

In [ ]:
def generate_answer(model, tokenizer, question, device='cuda'):
    inputs = tokenizer.encode_plus(question, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0]))
    return answer

def compute_metrics(references, predictions):
    bleu_scores = []
    rouge_l_scores = []
    f1_scores = []

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    for ref, pred in zip(references, predictions):
        # BLEU score
        bleu_score = sentence_bleu([ref.split()], pred.split())
        bleu_scores.append(bleu_score)

        # ROUGE-L score
        rouge_l_score = rouge.score(ref, pred)['rougeL'].fmeasure
        rouge_l_scores.append(rouge_l_score)

        # F1-score
        ref_tokens = set(ref.split())
        pred_tokens = set(pred.split())
        common_tokens = ref_tokens.intersection(pred_tokens)

        if len(common_tokens) == 0:
            f1 = 0.0
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(ref_tokens)
            f1 = 2 * (precision * recall) / (precision + recall)
        f1_scores.append(f1)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    return avg_bleu, avg_rouge_l, avg_f1

# Example usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

questions = test.question
references = test.answer

predictions = [generate_answer(model, tokenizer, q, device) for q in questions]

avg_bleu, avg_rouge_l, avg_f1 = compute_metrics(references, predictions)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average F1 Score: {avg_f1}")